In [62]:
# Last update: 2021-01-04
# Imports:
import glob  # To loop t files
import os  # To find the current working directory
import ast # Strings to lists
import pandas as pd  # For DataFrame
import numpy as np  # For its arrays

In [20]:
# Read in data and convert labels to vector:
path = os.path.join(os.getcwd(), "responses")
all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f, index_col=None) for f in all_files))
df.reset_index(drop=True, inplace=True)  # Drop the original index
print(df.head())
print(df.tail())
print("Input data shape:", df.shape)

   LL_AttemptsForProgress_0  LL_AttemptsForProgress_1  \
0                       3.0                       1.0   
1                       1.0                       1.0   
2                       8.0                       1.0   
3                       NaN                       NaN   
4                       NaN                       NaN   

   LL_AttemptsForProgress_2  LL_AttemptsForProgress_3  \
0                       2.0                       2.0   
1                       1.0                       1.0   
2                       1.0                       1.0   
3                       NaN                       NaN   
4                       NaN                       NaN   

                   LL_CorrectKeyword                LL_DetectedCommands  \
0           [0, 0, 1, 1, 0, 1, 0, 1]           [2, 2, 0, 1, 1, 2, 0, 3]   
1                       [1, 1, 1, 1]                       [4, 5, 2, 3]   
2  [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]  [2, 1, 2, 1, 2, 1, 3, 0, 1, 4, 5]   
3             

C:\Users\emmav\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


In [22]:
# Remove all practice trials:
print("Nr. of trials including practice:", len(df.index))
df = df[df.practice == "no"]
print("Nr. of trials excluding practice:", len(df.index))


Nr. of trials including practice: 204
Nr. of trials excluding practice: 204


In [36]:
# Remove all the columns that are exactly the same for each of the rows:
clean_df = df.copy()
only_LL = df.copy()

for col in df.columns:
    if len(df[col].unique()) == 1 or col.startswith("time_") or col.startswith("response_"):
        clean_df = clean_df.drop(col, axis=1)
    if not col.startswith("LL_") and col != "subject_nr" and col != "order":
        only_LL = only_LL.drop(col, axis=1)
   
clean_df = clean_df.reset_index(drop=True)


In [37]:
print(clean_df.columns)
print(only_LL.columns)


Index(['LL_AttemptsForProgress_0', 'LL_AttemptsForProgress_1',
       'LL_AttemptsForProgress_2', 'LL_AttemptsForProgress_3',
       'LL_CorrectKeyword', 'LL_DetectedCommands', 'LL_ProgressTrialFailedAt',
       'LL_TimeITIEnd', 'LL_TimeITIStart', 'LL_TimeKeywordDetection',
       'LL_TimeStartListening', 'LL_TimeStopListening', 'LL_TimeTrialEnd',
       'LL_TimeTrialStart', 'correct_practiceInstructionsGestural',
       'correct_practiceInstructionsVocal', 'count_blockLoop', 'count_cleanUp',
       'count_finishedBlockNotice', 'count_finishedPracticeGestural',
       'count_finishedPracticeVocal', 'count_interfaceSequence',
       'count_logTrial', 'count_practiceInstructionsGestural',
       'count_practiceInstructionsVocal', 'count_practiceLoop',
       'count_runTrial', 'count_setUpInterface', 'count_trialSequence',
       'datetime', 'experiment_file', 'experiment_path', 'fullscreen',
       'interface', 'live_row', 'live_row_interfaceLoop_even',
       'live_row_interfaceLoop_odd

In [34]:
clean_df

,LL_AttemptsForProgress_0,LL_AttemptsForProgress_1,LL_AttemptsForProgress_2,LL_AttemptsForProgress_3,LL_CorrectKeyword,LL_DetectedCommands,LL_ProgressTrialFailedAt,LL_TimeITIEnd,LL_TimeITIStart,LL_TimeKeywordDetection,...,interface,live_row,live_row_interfaceLoop_even,live_row_interfaceLoop_odd,logfile,opensesame_version,order,response,subject_nr,subject_parity
0,NaN,NaN,NaN,NaN,[],[],NaN,1.609196e+09,1.609196e+09,[],...,gestural,0,0.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[0, 3, 2, 1]",return,0,even
1,NaN,NaN,NaN,NaN,[],[],NaN,1.609196e+09,1.609196e+09,[],...,gestural,1,0.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[3, 1, 0, 5]",return,0,even
2,NaN,NaN,NaN,NaN,[],[],NaN,1.609196e+09,1.609196e+09,[],...,gestural,2,0.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[5, 3, 0, 4]",return,0,even
3,NaN,NaN,NaN,NaN,[0],[2],NaN,1.609196e+09,1.609196e+09,[1609195547.0030756],...,gestural,3,0.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[0, 3, 4, 2]",return,0,even
4,1.0,1.0,1.0,1.0,"[1, 1, 1, 1]","[5, 1, 2, 3]",NaN,1.609196e+09,1.609196e+09,"[1609195569.2287478, 1609195581.9229696, 16091...",...,gestural,4,0.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[5, 1, 2, 3]",return,0,even
5,1.0,2.0,1.0,1.0,"[1, 0, 1, 1, 1]","[3, 1, 0, 4, 2]",NaN,1.609196e+09,1.609196e+09,"[1609195627.9256763, 1609195640.3439653, 16091...",...,gestural,5,0.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[3, 0, 4, 2]",return,0,even
6,1.0,1.0,1.0,1.0,"[1, 1, 1, 1]","[3, 1, 0, 5]",NaN,1.609196e+09,1.609196e+09,"[1609195916.2044234, 1609195927.1323252, 16091...",...,vocal,0,1.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[3, 1, 0, 5]",return,0,even
7,1.0,1.0,NaN,NaN,"[1, 1]","[3, 4]",NaN,1.609196e+09,1.609196e+09,"[1609195964.3211377, 1609195975.4454372]",...,vocal,1,1.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[3, 4, 2, 1]",return,0,even
8,1.0,1.0,1.0,NaN,"[1, 1, 1]","[4, 1, 0]",NaN,1.609196e+09,1.609196e+09,"[1609195996.8739624, 1609196007.6801827, 16091...",...,vocal,2,1.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[4, 1, 0, 2]",return,0,even
9,NaN,NaN,NaN,NaN,[],[],NaN,1.609196e+09,1.609196e+09,[],...,vocal,3,1.0,NaN,C:/Users/Heleen/Documents/Intelligent technolo...,3.3.6,"[0, 2, 3, 5]",return,0,even


In [63]:
only_LL["LL_ProgressTrialFailedAt"].values
# print(only_LL.iloc[0]["LL_CorrectKeyword"])
# print(only_LL.iloc[47])
# print(only_LL.iloc[48])
# print(only_LL.iloc[52])

for i, correct_keyword in enumerate(only_LL["LL_CorrectKeyword"]):
    correct_keyword = ast.literal_eval(correct_keyword)  # Convert string to list
    # Now update the LL_AttemptsForProgress_X accordingly, and also LL_ProgressTrialFailedAt


<class 'list'>
[]
<class 'list'>
[]
<class 'list'>
[]
<class 'list'>
[0]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 0, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1]
<class 'list'>
[1, 1, 1]
<class 'list'>
[]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 0, 1]
<class 'list'>
[1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1]
<class 'list'>
[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
<class 'list'>
[1, 1, 1, 1]
